In [1]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [10]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [12]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [5]:
course_data = pd.read_sql("SELECT * FROM course", conn)
course_data.head()

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [12]:
course_data = course_data.rename(columns={"ID_Course": "ID_Course__c", "CourseCode":"CourseCode__c", "CourseName":"Name",
                            "CreditHours":"CreditHours__c", "BootCampCourse":"BootCampCourse__c"})
course_data.head()

,ID_Course__c,CourseCode__c,Name,CreditHours__c,BootCampCourse__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [13]:
course_load = course_data.to_dict('records')
course_load

[{'ID_Course__c': 1,
  'CourseCode__c': 'BC-DATAVIZ',
  'Name': 'Data Visualization and Analytics',
  'CreditHours__c': 12,
  'BootCampCourse__c': 1},
 {'ID_Course__c': 2,
  'CourseCode__c': 'BC-WEBDEV',
  'Name': 'Full Stack Web Development',
  'CreditHours__c': 12,
  'BootCampCourse__c': 1},
 {'ID_Course__c': 3,
  'CourseCode__c': 'BC-UIUX',
  'Name': 'User Interface/User Experience',
  'CreditHours__c': 12,
  'BootCampCourse__c': 1},
 {'ID_Course__c': 4,
  'CourseCode__c': 'CIS-349',
  'Name': 'Introduction to Databases',
  'CreditHours__c': 5,
  'BootCampCourse__c': 0},
 {'ID_Course__c': 5,
  'CourseCode__c': 'CIS-405',
  'Name': 'Database Programming',
  'CreditHours__c': 5,
  'BootCampCourse__c': 0},
 {'ID_Course__c': 6,
  'CourseCode__c': 'CIS-438',
  'Name': 'Database Administration',
  'CreditHours__c': 5,
  'BootCampCourse__c': 0},
 {'ID_Course__c': 7,
  'CourseCode__c': 'CIS-430',
  'Name': 'Business Systems Programming I',
  'CreditHours__c': 5,
  'BootCampCourse__c': 0},
 

In [14]:
try:
    sf.bulk.Course__c.insert(course_load)
except Exception as e:
    print(e)

## Moving over Student Data

In [308]:
student_data = pd.read_sql("SELECT * FROM student", conn)
student_data.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


In [309]:
student_data['FullName'] = student_data['FirstName'].str.cat(student_data['MiddleName'], sep=" ")
student_data['FullName'] = student_data['FullName'].str.cat(student_data['LastName'], sep = " ")
student_data = student_data.rename(columns={"ID_Student": "ID_Student__c", "StudentID":"StudentID__c", "LastName":"LastName__c",
                            "FirstName":"FirstName__c", "MiddleName":"MiddleName__c",
                                "BirthDate":"BirthDate__c", "Gender":"Gender__c", "FullName": "Name"})
student_data.head()
                                                             

,ID_Student__c,StudentID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c,Gender__c,Name
0,33,25004961,Moore,Heather,Alice,None,F,Heather Alice Moore
1,34,25003514,Multak,Ilana,Cecille,None,F,Ilana Cecille Multak
2,35,25005833,Murillo,Jessica,Dorothy,None,F,Jessica Dorothy Murillo
3,36,25002589,Romanowski,Kandra,Genevieve,None,F,Kandra Genevieve Romanowski
4,37,25007185,Hoffer,Katherine,Lynnette,None,F,Katherine Lynnette Hoffer


In [310]:
student_load = student_data.to_dict('records')

In [311]:
try:
    sf.bulk.Student__c.insert(student_load)
except Exception as e:
    print(e)

## Moving over Staff Data

In [20]:
staff_data = pd.read_sql("SELECT * FROM staff", conn)
staff_data

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [21]:
staff_data['FullName'] = staff_data['FirstName'].str.cat(staff_data['MiddleName'], sep=" ")
staff_data['FullName'] = staff_data['FullName'].str.cat(staff_data['LastName'], sep = " ")

staff_data = staff_data.rename(columns={"ID_Staff": "ID_Staff__c", "EmployeeID":"EmployeeID__c", "LastName":"LastName__c",
                            "FirstName":"FirstName__c", "MiddleName":"MiddleName__c",
                                "BirthDate":"BirthDate__c", "FullName":'Name'})
staff_data.head()


,ID_Staff__c,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c,Name
0,1,000184220,Wimberly,Sam,Nico,1995-07-05,Sam Nico Wimberly
1,2,000130109,Sanford,Gemini,Blair,1992-04-22,Gemini Blair Sanford
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21,Dartanion De Angelo Williams
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07,Sarah Nisan Popelka


In [23]:
for index, row in staff_data.iterrows():

    record = {
        'Name': row['Name'],
        'FirstName__c': row['FirstName__c'],
        'MiddleName__c':row['MiddleName__c'],
        'LastName__c':row['LastName__c'],
        'ID_Staff__c':row['ID_Staff__c'],
        'EmployeeID__c':row['EmployeeID__c'],
        'BirthDate__c': row['BirthDate__c'].isoformat()
    }
    
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)

## Moving over Class Info

In [13]:
class_data = pd.read_sql("SELECT * FROM class", conn)
class_data

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [14]:
query = sf.query("SELECT ID_Course__c FROM Course__c")
query

OrderedDict([('totalSize', 8),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Course__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Course__c/a033h000006VgblAAC')])),
                            ('ID_Course__c', 1.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Course__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Course__c/a033h000006VgbmAAC')])),
                            ('ID_Course__c', 2.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Course__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Course__c/a033h000006VgbnAAC')])),
        

In [15]:
course_id_dict = {}
i = 0
while i < len(query['records']):
    url = query['records'][i]['attributes']['url'].split('/')[6]
    course_id = int(query['records'][i]['ID_Course__c'])
    course_id_dict[course_id]=url
    i = i+1
print(course_id_dict)

a033h000006VgblAAC 1
a033h000006VgbmAAC 2
a033h000006VgbnAAC 3
a033h000006VgboAAC 4
a033h000006VgbpAAC 5
a033h000006VgbqAAC 6
a033h000006VgbsAAC 8
a033h000006VgbrAAC 7
{1: 'a033h000006VgblAAC', 2: 'a033h000006VgbmAAC', 3: 'a033h000006VgbnAAC', 4: 'a033h000006VgboAAC', 5: 'a033h000006VgbpAAC', 6: 'a033h000006VgbqAAC', 8: 'a033h000006VgbsAAC', 7: 'a033h000006VgbrAAC'}


In [18]:
class_data['ID_Course'] = class_data['ID_Course'].apply(lambda x: course_id_dict[x])
class_data

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,a033h000006VgblAAC,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,a033h000006VgblAAC,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,a033h000006VgbmAAC,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,a033h000006VgbnAAC,GWARL201905UIUX3,2019-05-14,2019-11-07


In [8]:
#for index, row in class_data.iterrows():
    #class_data['ID_Course'][index] = course_id_dict[row['ID_Course']]
#class_data

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,a033h000006VgblAAC,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,a033h000006VgblAAC,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,a033h000006VgbmAAC,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,a033h000006VgbnAAC,GWARL201905UIUX3,2019-05-14,2019-11-07


In [12]:
for index, row in class_data.iterrows():

    record = {
        'ID_Class__c': row['ID_Class'],
        'ID_Course__c':row['ID_Course'],
        'Name': row['Section'],
        'Section__c':row['Section'],
        'StartDate__c':row['StartDate'].isoformat(),
        'EndDate__c':row['EndDate'].isoformat()
    }
    
    try:
        sf.Class__c.create(record)
    except Exception as e:
        print(e)

## Moving over Staff Assignment Info

In [19]:
staffa_data = pd.read_sql("SELECT * FROM staffassignment", conn)
staffa_data

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [20]:
query2 = sf.query("SELECT ID_Staff__c FROM Staff__c")
query2

OrderedDict([('totalSize', 4),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Staff__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Staff__c/a0B3h000001FtVAEA0')])),
                            ('ID_Staff__c', 1.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Staff__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Staff__c/a0B3h000001FtVKEA0')])),
                            ('ID_Staff__c', 3.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Staff__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Staff__c/a0B3h000001FtVPEA0')])),
                

In [21]:
staff_id_dict = {}
i = 0
while i < len(query2['records']):
    url = query2['records'][i]['attributes']['url'].split('/')[6]
    staff_id = int(query2['records'][i]['ID_Staff__c'])
    print(url)
    staff_id_dict[staff_id]=url
    i = i+1
print(staff_id_dict)

a0B3h000001FtVAEA0
a0B3h000001FtVKEA0
a0B3h000001FtVPEA0
a0B3h000001FtVFEA0
{1: 'a0B3h000001FtVAEA0', 3: 'a0B3h000001FtVKEA0', 4: 'a0B3h000001FtVPEA0', 2: 'a0B3h000001FtVFEA0'}


In [23]:
staffa_data['ID_Staff'] = staffa_data['ID_Staff'].apply(lambda x: staff_id_dict[x])
staffa_data

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,a0B3h000001FtVAEA0,1,Teacher Assistant,2020-03-16,None
1,2,a0B3h000001FtVPEA0,1,Teacher Assistant,2020-03-16,None
2,3,a0B3h000001FtVKEA0,1,Instructor,2020-03-16,None


In [27]:
#for index, row in staffa_data.iterrows():
    #staffa_data['ID_Staff'][index] = staff_id_dict[row['ID_Staff']]
#staffa_data

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,a0B3h000001FtVAEA0,1,Teacher Assistant,2020-03-16,None
1,2,a0B3h000001FtVPEA0,1,Teacher Assistant,2020-03-16,None
2,3,a0B3h000001FtVKEA0,1,Instructor,2020-03-16,None


In [24]:
query3 = sf.query("SELECT ID_Class__c FROM Class__c")
query3

OrderedDict([('totalSize', 4),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Class__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Class__c/a0A3h000003OMvEEAW')])),
                            ('ID_Class__c', 3.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Class__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Class__c/a0A3h000003OMv4EAG')])),
                            ('ID_Class__c', 1.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Class__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Class__c/a0A3h000003OMvJEAW')])),
                

In [25]:
class_id_dict = {}
i = 0
while i < len(query3['records']):
    url = query3['records'][i]['attributes']['url'].split('/')[6]
    class_id = int(query3['records'][i]['ID_Class__c'])
    print(url)
    class_id_dict[class_id]=url
    i = i+1
print(class_id_dict)

a0A3h000003OMvEEAW
a0A3h000003OMv4EAG
a0A3h000003OMvJEAW
a0A3h000003OMv9EAG
{3: 'a0A3h000003OMvEEAW', 1: 'a0A3h000003OMv4EAG', 4: 'a0A3h000003OMvJEAW', 2: 'a0A3h000003OMv9EAG'}


In [26]:
staffa_data['ID_Class'] = staffa_data['ID_Class'].apply(lambda x: class_id_dict[x])
staffa_data

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,a0B3h000001FtVAEA0,a0A3h000003OMv4EAG,Teacher Assistant,2020-03-16,None
1,2,a0B3h000001FtVPEA0,a0A3h000003OMv4EAG,Teacher Assistant,2020-03-16,None
2,3,a0B3h000001FtVKEA0,a0A3h000003OMv4EAG,Instructor,2020-03-16,None


In [30]:
#for index, row in staffa_data.iterrows():
    #staffa_data['ID_Class'][index] = class_id_dict[row['ID_Class']]
#staffa_data

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,a0B3h000001FtVAEA0,a0A3h000003OMv4EAG,Teacher Assistant,2020-03-16,None
1,2,a0B3h000001FtVPEA0,a0A3h000003OMv4EAG,Teacher Assistant,2020-03-16,None
2,3,a0B3h000001FtVKEA0,a0A3h000003OMv4EAG,Instructor,2020-03-16,None


In [31]:
for index, row in staffa_data.iterrows():

    record = {
        'ID_StaffAssignment__c': row['ID_StaffAssignment'],
        'ID_Class__c': row['ID_Class'],
        'ID_Staff__c':row['ID_Staff'],
        'Role__c':row['Role'],
        'StartDate__c':row['StartDate'].isoformat(),
        'EndDate__c':row['EndDate']
    }
    
    try:
        sf.StaffAssignment__c.create(record)
    except Exception as e:
        print(e)

## Moving over Class Participant Data

In [27]:
classp_data = pd.read_sql("SELECT * FROM classparticipant", conn)
classp_data.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None


In [28]:
query4 = sf.query("SELECT ID_Student__c FROM Student__c")
query4

OrderedDict([('totalSize', 30),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Student__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Student__c/a0E3h000000aquBEAQ')])),
                            ('ID_Student__c', 33.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Student__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Student__c/a0E3h000000aquCEAQ')])),
                            ('ID_Student__c', 34.0)]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Student__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Student__c/a0E3h000000aquDEAQ')])

In [29]:
student_id_dict = {}
i = 0
while i < len(query4['records']):
    url = query4['records'][i]['attributes']['url'].split('/')[6]
    student_id = query4['records'][i]['ID_Student__c']
    student_id_dict[student_id]=url
    i = i+1
print(student_id_dict)

{33.0: 'a0E3h000000aquBEAQ', 34.0: 'a0E3h000000aquCEAQ', 35.0: 'a0E3h000000aquDEAQ', 36.0: 'a0E3h000000aquEEAQ', 37.0: 'a0E3h000000aquFEAQ', 38.0: 'a0E3h000000aquGEAQ', 39.0: 'a0E3h000000aquHEAQ', 40.0: 'a0E3h000000aquIEAQ', 41.0: 'a0E3h000000aquJEAQ', 42.0: 'a0E3h000000aquKEAQ', 43.0: 'a0E3h000000aquLEAQ', 44.0: 'a0E3h000000aquMEAQ', 45.0: 'a0E3h000000aquNEAQ', 46.0: 'a0E3h000000aquOEAQ', 47.0: 'a0E3h000000aquPEAQ', 48.0: 'a0E3h000000aquQEAQ', 49.0: 'a0E3h000000aquREAQ', 50.0: 'a0E3h000000aquSEAQ', 51.0: 'a0E3h000000aquTEAQ', 52.0: 'a0E3h000000aquUEAQ', 53.0: 'a0E3h000000aquVEAQ', 54.0: 'a0E3h000000aquWEAQ', 55.0: 'a0E3h000000aquXEAQ', 56.0: 'a0E3h000000aquYEAQ', 57.0: 'a0E3h000000aquZEAQ', 58.0: 'a0E3h000000aquaEAA', 59.0: 'a0E3h000000aqubEAA', 60.0: 'a0E3h000000aqucEAA', 61.0: 'a0E3h000000aqudEAA', 62.0: 'a0E3h000000aqueEAA'}


In [30]:
classp_data['ID_Student'] = classp_data['ID_Student'].apply(lambda x: student_id_dict[x])
classp_data

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,a0E3h000000aquBEAQ,1,2020-03-16,None
1,2,a0E3h000000aquCEAQ,1,2020-03-16,None
2,3,a0E3h000000aquDEAQ,1,2020-03-16,None
3,4,a0E3h000000aqueEAA,1,2020-03-16,None
4,5,a0E3h000000aquEEAQ,1,2020-03-16,None
5,6,a0E3h000000aquFEAQ,1,2020-03-16,None
6,7,a0E3h000000aquGEAQ,1,2020-03-16,None
7,8,a0E3h000000aquHEAQ,1,2020-03-16,None
8,9,a0E3h000000aquIEAQ,1,2020-03-16,None
9,10,a0E3h000000aquJEAQ,1,2020-03-16,None


In [35]:
#for index, row in classp_data.iterrows():
    #classp_data['ID_Student'][index] = student_id_dict[row['ID_Student']]
#classp_data

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,a0E3h000000aquBEAQ,1,2020-03-16,None
1,2,a0E3h000000aquCEAQ,1,2020-03-16,None
2,3,a0E3h000000aquDEAQ,1,2020-03-16,None
3,4,a0E3h000000aqueEAA,1,2020-03-16,None
4,5,a0E3h000000aquEEAQ,1,2020-03-16,None
5,6,a0E3h000000aquFEAQ,1,2020-03-16,None
6,7,a0E3h000000aquGEAQ,1,2020-03-16,None
7,8,a0E3h000000aquHEAQ,1,2020-03-16,None
8,9,a0E3h000000aquIEAQ,1,2020-03-16,None
9,10,a0E3h000000aquJEAQ,1,2020-03-16,None


In [31]:
classp_data['ID_Class'] = classp_data['ID_Class'].apply(lambda x: class_id_dict[x])
classp_data.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,a0E3h000000aquBEAQ,a0A3h000003OMv4EAG,2020-03-16,None
1,2,a0E3h000000aquCEAQ,a0A3h000003OMv4EAG,2020-03-16,None
2,3,a0E3h000000aquDEAQ,a0A3h000003OMv4EAG,2020-03-16,None
3,4,a0E3h000000aqueEAA,a0A3h000003OMv4EAG,2020-03-16,None
4,5,a0E3h000000aquEEAQ,a0A3h000003OMv4EAG,2020-03-16,None


In [36]:
#for index, row in classp_data.iterrows():
    #classp_data['ID_Class'][index] = class_id_dict[row['ID_Class']]
#classp_data

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,a0E3h000000aquBEAQ,a0A3h000003OMv4EAG,2020-03-16,None
1,2,a0E3h000000aquCEAQ,a0A3h000003OMv4EAG,2020-03-16,None
2,3,a0E3h000000aquDEAQ,a0A3h000003OMv4EAG,2020-03-16,None
3,4,a0E3h000000aqueEAA,a0A3h000003OMv4EAG,2020-03-16,None
4,5,a0E3h000000aquEEAQ,a0A3h000003OMv4EAG,2020-03-16,None
5,6,a0E3h000000aquFEAQ,a0A3h000003OMv4EAG,2020-03-16,None
6,7,a0E3h000000aquGEAQ,a0A3h000003OMv4EAG,2020-03-16,None
7,8,a0E3h000000aquHEAQ,a0A3h000003OMv4EAG,2020-03-16,None
8,9,a0E3h000000aquIEAQ,a0A3h000003OMv4EAG,2020-03-16,None
9,10,a0E3h000000aquJEAQ,a0A3h000003OMv4EAG,2020-03-16,None


In [37]:
for index, row in classp_data.iterrows():

    record = {
        'ID_ClassParticipant__c': row['ID_ClassParticipant'],
        'ID_Class__c': row['ID_Class'],
        'ID_Student__c':row['ID_Student'],
        'StartDate__c':row['StartDate'].isoformat(),
        'EndDate__c':row['EndDate']
    }
    
    try:
        sf.classparticipant__c.create(record)
    except Exception as e:
        print(e)